# ClipCard Field Evaluation: Industrial/Aviation
**Purpose:** This notebook analyzes historical data from the `clipmap` CSVs to evaluate the real-world performance of the ClipCard system over time for the **Industrial/Aviation** silo.
It calculates and visualizes key metrics as defined in the `dfm-industrial-aviation.md` framework document, such as:- Deviation Incidents- Closeout On-Time %- Near Miss Rate

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport matplotlib.dates as mdates# Load the aggregated ClipCard datatry:    cards_df = pd.read_csv('../../data/clipmap/cards.csv')    events_df = pd.read_csv('../../data/clipmap/events.csv')    # Filter for the correct domain and merge    industrial_cards = cards_df[cards_df['domain'] == 'dfm-industrial-aviation'].copy()    merged_df = pd.merge(industrial_cards, events_df, left_on='id', right_on='card_id')    merged_df['time'] = pd.to_datetime(merged_df['time'])except FileNotFoundError as e:    print(f'ERROR: {e.filename} not found. Please ensure the data exists.')    merged_df = pd.DataFrame()

### Metric 1: Deviation Incidents
**Definition:** Incidents during temporary deviations (per 1k operating hours or flight cycles).**Goal:** Decrease vs. baseline.
**Note:** This metric requires logging operating hours/cycles and specific 'incident' event types. We will use 'Incident Prevented' as a proxy for an incident that was caught, and generate synthetic data for operating hours.

In [ ]:
if not merged_df.empty:    # Group by week for trend analysis    events_by_week = merged_df.set_index('time').resample('W')        # --- Placeholder Data Generation ---    placeholder_df = events_by_week.size().to_frame(name='total_events')    # Assume 'Incident Prevented' is our proxy for a deviation incident that was caught    placeholder_df['deviation_incidents'] = events_by_week['outcome'].apply(lambda x: (x == 'Incident Prevented').sum())    # Assume a constant 1000 operating hours per week for rate calculation    placeholder_df['op_hours'] = 1000     placeholder_df['incident_rate'] = (placeholder_df['deviation_incidents'] / placeholder_df['op_hours'] * 1000).fillna(0)    # --- End Placeholder ---    # Plotting    fig, ax = plt.subplots(figsize=(12, 6))    ax.plot(placeholder_df.index, placeholder_df['incident_rate'], marker='o', linestyle='-', label='Deviation Incident Rate (per 1k hours)')    ax.set_title('Deviation Incident Rate Over Time (Synthetic Data)')    ax.set_ylabel('Rate (per 1k hours)')    ax.set_xlabel('Week')    ax.grid(True, which='both', linestyle='--', linewidth=0.5)    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))    plt.xticks(rotation=45)    ax.legend()    plt.tight_layout()    plt.show()else:    print('Data is empty. Cannot calculate Deviation Incident Rate.')

### Metric 2: Closeout On-Time %
**Definition:** Share of MOC deviations closed by planned demobilization time.**Goal:** ≥85%.
**Note:** This requires tracking the planned closeout time vs. the actual closeout event time. We will generate synthetic data for this.

In [ ]:
if not merged_df.empty:    events_by_week = merged_df.set_index('time').resample('W')        # --- Placeholder Data Generation ---    placeholder_df = events_by_week.size().to_frame(name='total_moc_closeouts')    # Assume an improving trend of on-time closeouts    on_time_proportion = 0.75 + (np.arange(len(placeholder_df)) * 0.05)    on_time_proportion[on_time_proportion > 0.95] = 0.95 # Cap at 95%    placeholder_df['on_time_closeouts'] = (placeholder_df['total_moc_closeouts'] * on_time_proportion).astype(int)    placeholder_df['on_time_rate'] = (placeholder_df['on_time_closeouts'] / placeholder_df['total_moc_closeouts'] * 100).fillna(0)    # --- End Placeholder ---        # Plotting    fig, ax = plt.subplots(figsize=(12, 6))    ax.plot(placeholder_df.index, placeholder_df['on_time_rate'], marker='s', linestyle='--', color='green', label='Closeout On-Time %')    ax.axhline(85, color='red', linestyle=':', label='Goal (85%)')    ax.set_title('MOC Closeout On-Time Rate Over Time (Synthetic Data)')    ax.set_ylabel('On-Time Rate (%)')    ax.set_xlabel('Week')    ax.grid(True, which='both', linestyle='--', linewidth=0.5)    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))    ax.set_ylim(bottom=50, top=105)    plt.xticks(rotation=45)    ax.legend()    plt.tight_layout()    plt.show()else:    print('Data is empty. Cannot calculate Closeout On-Time %.')